# AST 304 Project 2 Writeup

Elias Taira, ...., ... ,....

In [1]:
import numpy as np
import pandas as pd
from structure import integrate
from eos import density
import matplotlib.pyplot as plt
import scipy.optimize as op

### Defining Physical Constants

For our code, we define all of our physical and astronomical constants in the astro_const.py file. However, since we do not need to use all of the constants contained within the file for this writup, we will only be importing the values that we need.

In [2]:
from astro_const import Msun, Rsun, G

### Implementing the Equation of State

For the equations of state that we will be using to help us in our integration, we store them within the *eos.py* file within the functions *pressure* and *density*, which after taking in either the density or pressure depending on which function is being called, as well as the $\mu_e$ (which we have decided to set to 2 for this assignment), will output the electron degeneracy pressure (in pascals) for pressure and the mass density (in $\frac{kg}{m^3}$) for the density function.

To validate these functions we ran the *test_eos.py* file which takes in an input of our *eos.py* file and validates that it is working properly.

In [3]:
! python ./test_eos.py ./eos.py

Exception ignored in: <function Socket.__del__ at 0x0000015BAD64DF70>
Traceback (most recent call last):
  File "c:\Users\elias\anaconda3\lib\site-packages\zmq\sugar\socket.py", line 97, in __del__
Exception ignored in: <_io.FileIO name=4 mode='wb' closefd=True>
Traceback (most recent call last):
  File "c:\Users\elias\anaconda3\lib\site-packages\IPython\utils\_process_win32.py", line 124, in system
        warnings.warn(
return process_handler(cmd, _system_body)
Exception ignored in: <_io.FileIO name=5 mode='rb' closefd=True>
Traceback (most recent call last):
  File "c:\Users\elias\anaconda3\lib\site-packages\IPython\utils\_process_win32.py", line 124, in system
    return process_handler(cmd, _system_body)
Exception ignored in: <_io.FileIO name=6 mode='rb' closefd=True>
Traceback (most recent call last):
  File "c:\Users\elias\anaconda3\lib\site-packages\IPython\utils\_process_win32.py", line 124, in system
    return process_handler(cmd, _system_body)


Comparing EOS to eos_table.txt...

   rho (table)    rho (test)    difference       P (table)      P (test)    difference

  1.000000e+05  1.000000e+05  5.820766e-16    6.810445e+14  6.810445e+14  0.000000e+00
  1.123655e+05  1.123655e+05  2.590104e-16    8.271108e+14  8.271108e+14  7.556424e-16
  1.262600e+05  1.262600e+05  1.037282e-15    1.004504e+15  1.004504e+15 -7.466368e-16
  1.418727e+05  1.418727e+05  6.154215e-16    1.219944e+15  1.219944e+15  2.049274e-16
  1.594159e+05  1.594159e+05  1.825654e-16    1.481591e+15  1.481591e+15  6.749502e-16
  1.791284e+05  1.791284e+05  6.498986e-16    1.799353e+15  1.799353e+15 -1.389388e-16
  2.012785e+05  2.012785e+05  7.229740e-16    2.185268e+15  2.185268e+15 -3.432074e-16
  2.261676e+05  2.261676e+05  5.147303e-16    2.653950e+15  2.653950e+15  1.883984e-16
  2.541343e+05  2.541343e+05  5.726073e-16    3.223153e+15  3.223153e+15  0.000000e+00
  2.855592e+05  2.855592e+05  8.153497e-16    3.914436e+15  3.914436e+15 -3.831970e-16
  3.208

After running the above cell, we find that our equations of state are working as they should

### Developing the Integration Method

Now onto the step in which we run our integration. For this, we utilized the many functions stored within the *structure.py* file. The first function within the file, *stellar_derivatives*, takes in some mass, pressure, radius and $\mu_e$ and then outputs the derivatives of radius and pressure respectively. This function within the integrate function. Next we have the *Central_values* function which essentially just assist in setting up the initial conditions. The *lengthscales* function provides a variable stepsize to ensure that we do not integrate over areas outside of the star. Finally, we have the *integration* function where we actually perform the integration to obtain values for the mass, radius and pressure of the white dwarf as we iterate from the center to the edge of the dwarf.

### Building the Mass-Radius Table

In [3]:
def mass_finder(Pc, wanted_mass, delta_m, eta, xi, mue):

    m_step, r_step, p_step = integrate(Pc,delta_m,eta,xi,mue)

    return wanted_mass - m_step[-1]

mass_array = np.arange(0.1, 1.1, 0.1)

# Setting some initial conditions
mue = 2

eta = 1e-10

xi = 0.05

r_array = np.zeros_like(mass_array)

Pc_array = np.zeros_like(mass_array)

Pc_mod_array = np.zeros_like(mass_array)

rho_array = np.zeros_like(mass_array)

rho_mod_array = np.zeros_like(mass_array)

for i in range(len(mass_array)):

    # converting mass to units of kg
    Smass = Msun * mass_array[i]
    
    delta_m = Smass*1e-6

    desired_pressure = op.bisect(mass_finder, a = 1e19, b = 1e23, xtol = 1e-10, args = (Smass, delta_m, eta, xi, mue))

    m, r, p = integrate(desired_pressure,delta_m,eta,xi,mue)

    r_array[i] = r[-1]/Msun

    Pc_array[i] = desired_pressure

    Pc_mod_array[i] = desired_pressure/(G*Smass**2*r[-1]**(-4))

    rho_array[i] = density(desired_pressure, mue)

    rho_mod_array[i] = density(desired_pressure, mue)/(3*Smass/(4*np.pi*r[-1]**3))


In [13]:
mr_table = pd.DataFrame()

mr_table[r"$\frac{M}/{M_\dot}$"] = mass_array

mr_table[r"$\frac{R}{R_\dot}"] = r_array

mr_table[r"Pc"] = Pc_array

mr_table[r"Pc_mod"] = Pc_mod_array

mr_table[r"\rho"] = rho_array

mr_table[r"\rho_mod"] = rho_mod_array

mr_table


,$\frac{M}/{M_\dot}$,$\frac{R}{R_\dot},Pc,Pc_mod,\rho,\rho_mod
0,0.1,9.618683e-24,1.517726e+19,0.769594,4.062662e+07,5.987671
1,0.2,7.634354e-24,1.529771e+20,0.769594,1.625065e+08,5.987671
2,0.3,6.669222e-24,5.910135e+20,0.769594,3.656395e+08,5.987671
3,0.4,6.059391e-24,1.541913e+21,0.769594,6.500258e+08,5.987671
4,0.5,5.625040e-24,3.244093e+21,0.769594,1.015665e+09,5.987671
5,0.6,5.293365e-24,5.957043e+21,0.769594,1.462558e+09,5.987671
6,0.7,5.028243e-24,9.958334e+21,0.769594,1.990704e+09,5.987671
7,0.8,4.809341e-24,1.554151e+22,0.769594,2.600103e+09,5.987671
8,0.9,4.624180e-24,2.301450e+22,0.769594,3.290756e+09,5.987671
9,1.0,4.464597e-24,3.269841e+22,0.769594,4.062662e+09,5.987671
